In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

In [ ]:
import MetaTrader5 as mt5
# from vectorbt.portfolio import Portfolio as vbt_Portfolio

# Connection

In [ ]:
# RICO_REAL   = { 'login':16331371,   'password':'#000000Zz', 'server':'Rico-PRD'                 }
# CLEAR_REAL  = { 'login':1018156111, 'password':'#000000Zz', 'server':'ClearInvestimentos-CLEAR' }

# PATH_5 = r'C:\Program Files\MetaTrader 5 _5\terminal64.exe'
# PATH_7 = r'C:\Program Files\MetaTrader 5 _7\terminal64.exe'

# assert mt5.initialize(PATH_5, **CLEAR_REAL)

# Variables

In [ ]:
SYMBOLS = ['WIN$'] # ['WDO$']
TFRAMES = ['M5','M15','H1','H4','D1']

# Data

In [ ]:
# def MT5_LOAD_DATA(SYMBOLS, TFRAMES, start_pos=0, count=99_999):
#     Pipe = []
#     for (symbol, TF) in it.product(SYMBOLS, TFRAMES):
#         timeframe = { 'M1':mt5.TIMEFRAME_M1, 'M5':mt5.TIMEFRAME_M5, 'M10':mt5.TIMEFRAME_M10, 'M15':mt5.TIMEFRAME_M15, 'M20':mt5.TIMEFRAME_M20, 'M30':mt5.TIMEFRAME_M30, 'H1':mt5.TIMEFRAME_H1, 'H2':mt5.TIMEFRAME_H2, 'H3':mt5.TIMEFRAME_H3, 'H4':mt5.TIMEFRAME_H4, 'H6':mt5.TIMEFRAME_H6, 'H8':mt5.TIMEFRAME_H8, 'H12':mt5.TIMEFRAME_H12, 'D1':mt5.TIMEFRAME_D1, 'W1':mt5.TIMEFRAME_W1 }[TF]
#         Raw       = pd.DataFrame(mt5.copy_rates_from_pos(symbol, timeframe, start_pos, count))
#         Raw.insert(0, 'a',          Raw.index + 1)
#         Raw.insert(1, 'z',          Raw.index - Raw.index.max())
#         Raw.insert(2, 'symbol',     symbol)
#         Raw.insert(3, 'tf',         TF)
#         Raw.insert(4, 'datetime',   pd.to_datetime(Raw['time'], unit='s'))
#         Pipe.append(Raw)
#     pass
#     return pd.concat(Pipe, ignore_index=1)

# Raw = MT5_LOAD_DATA(SYMBOLS, TFRAMES)
# Raw.head()

In [ ]:
# Raw.to_csv('Storage/Raw.csv', index=False)
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def FORMAT_SOURCE(Raw):
    Src = pd.DataFrame()
    Src['A']        = Raw['a']
    Src['Z']        = Raw['z']
    Src['Symbol']   = Raw['symbol']
    Src['TF']       = Raw['tf']
    
    Src['Datetime'] = Raw['datetime']
    Src['Date']     = Raw['datetime'].dt.date
    Src['Time']     = Raw['datetime'].dt.time
    
    Src['Ticks']    = Raw['tick_volume']
    Src['Volume']   = Raw['real_volume']
    Src['Price']    = Raw['close']
    
    Src['Open']     = Raw['open']
    Src['High']     = Raw['high']
    Src['Low']      = Raw['low']
    Src['Close']    = Raw['close']

    Src['Change']   = Src['Close'] - Src['Open']
    Src['HL']       = Src['High']  - Src['Low']
    return Src 

Src = FORMAT_SOURCE(Raw)
Src.head()

In [ ]:
# Src.groupby(['Symbol','TF'],sort=0)[['Datetime']].first()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)

    Calc['Chg Psc']         = Calc.groupby(['Symbol','TF'], sort=0)['Change'] .expanding(20).rank(ascending=1, pct=1).mul(100) .reset_index(drop=1)
    Calc['HL Psc']          = Calc.groupby(['Symbol','TF'], sort=0)['HL']     .expanding(20).rank(ascending=1, pct=1).mul(100) .reset_index(drop=1)
    
    Calc['Chg Norm']        = Calc.apply(lambda x: x['Change'] if (5 < x['Chg Psc'] < 95) else nan, axis=1)
    Calc['HL Norm']         = Calc.apply(lambda x: x['HL']     if (5 < x['HL Psc']  < 95) else nan, axis=1)

    Calc['Chg Norm Abs']    = Calc['Chg Norm'].abs()
    Calc['Chg Norm Pos']    = Calc['Chg Norm'].apply(lambda x: x if (x >= 0) else nan)
    Calc['Chg Norm Neg']    = Calc['Chg Norm'].apply(lambda x: x if (x <  0) else nan)
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [55]:
Calc.groupby(['Symbol','TF'], sort=0)[['Chg Norm']].describe().round(0).astype(int)

Chg Norm                                      
              count mean   std   min   25% 50%  75%   max
Symbol TF                                                
WIN$   M5     93460    0   100  -395   -62   0   63   374
       M15    42992   -1   193  -556  -118   0  118   557
       H1     11070   -5   394 -1237  -242   0  243  1042
       H4      3468   -5   740 -2092  -462  10  453  1892
       D1      1141  -42  1384 -3944 -1012  29  954  2983
WDO$   M5     92977    0     3   -11    -2   0    2     9
       M15    43258    0     7   -21    -4   0    4    22
       H1     11131    0    13   -51    -8   0    8    51
       H4      3491    0    25  -112   -15   0   14    74
       D1      1158    0    48  -138   -32  -1   31   119

In [56]:
Calc.groupby(['Symbol','TF'], sort=0)[['Chg Norm Abs']].describe().round(0).astype(int)

Chg Norm Abs                                     
                  count  mean  std min  25%  50%   75%   max
Symbol TF                                                   
WIN$   M5         93460    78   63   0   28   62   113   395
       M15        42992   150  121   0   53  118   219   557
       H1         11070   306  249   0  106  242   445  1237
       H4          3468   580  460   0  199  458   869  2092
       D1          1141  1127  804   0  463  972  1620  3944
WDO$   M5         92977     3    2   0    1    2     4    11
       M15        43258     5    4   0    2    4     8    22
       H1         11131    10    9   0    4    8    15    51
       H4          3491    19   16   0    6   15    28   112
       D1          1158    38   29   0   14   31    56   138

In [57]:
Calc.groupby(['Symbol','TF'], sort=0)[['Chg Norm Pos']].describe().round(0).astype(int)

Chg Norm Pos                                     
                  count  mean  std min  25%  50%   75%   max
Symbol TF                                                   
WIN$   M5         48284    76   63   0   26   60   111   374
       M15        21860   146  120   0   51  115   216   557
       H1          5604   298  241   0  101  239   432  1042
       H4          1763   565  451   0  194  441   849  1892
       D1           584  1060  755   0  436  937  1591  2983
WDO$   M5         50767     2    2   0    1    2     4     9
       M15        22676     5    4   0    2    4     7    22
       H1          5755    10    9   0    3    8    15    51
       H4          1765    19   16   0    7   14    27    74
       D1           578    38   30   0   13   31    57   119

In [58]:
Calc.groupby(['Symbol','TF'], sort=0)[['Chg Norm Neg']].describe().round(0).astype(int)

Chg Norm Neg                                      
                  count  mean  std   min   25%   50%  75% max
Symbol TF                                                    
WIN$   M5         45176   -81   63  -395  -116   -64  -31  -5
       M15        21132  -154  122  -556  -223  -120  -56  -5
       H1          5466  -314  256 -1237  -457  -247 -111  -5
       H4          1705  -595  470 -2092  -896  -470 -204  -5
       D1           557 -1197  847 -3944 -1650 -1049 -508  -6
WDO$   M5         42210    -3    2   -11    -4    -2   -1   0
       M15        20582    -5    4   -21    -8    -4   -2   0
       H1          5376   -11    9   -51   -16    -8   -4   0
       H4          1726   -20   17  -112   -29   -15   -6   0
       D1           580   -38   28  -138   -55   -32  -15   0

In [59]:
Calc.groupby(['Symbol','TF'], sort=0)[['HL Norm']].describe().round(0).astype(int)

HL Norm                                         
             count  mean  std   min   25%   50%   75%   max
Symbol TF                                                  
WIN$   M5    86774   203   95    71   130   178   254   697
       M15   39409   396  183   133   254   352   500  1046
       H1    10053   791  361   245   509   708  1004  1994
       H4     3206  1424  632   445   916  1305  1825  3524
       D1     1022  2605  899  1163  1909  2440  3145  5646
WDO$   M5    83722     7    3     2     4     6     9    18
       M15   38980    14    7     4     9    12    17    41
       H1     9904    27   13     8    17    25    35    74
       H4     3160    49   23    12    30    44    64   143
       D1     1009    91   33    37    66    85   111   196

In [60]:
raise Exception('STOP')

Exception: STOP

# Chart

In [ ]:
# for SYMBOL, Asset in Src.groupby('Symbol', sort=0):

#     plt.title(f'{SYMBOL} - Candle Size Distribuition', fontweight='bold')

#     for TF, Data in Asset.groupby('TF', sort=0): 
#         # sns.kdeplot(Data['Change'], label=TF)
#         # sns.histplot(Data['Change'], label=TF, stat='count', bins=100, element='step', fill=False)
#         plt.hist(Data['Change'], bins=100, label=TF, histtype='step')


#     plt.ylabel('Frequency')
#     plt.xlabel('Candle Size')

#     plt.yscale('log', base=10)
#     plt.gca().yaxis.set_major_formatter(mpl.ticker.ScalarFormatter())

#     plt.legend()
#     plt.tight_layout()
#     plt.show()